## Prompt Chaining

In [1]:
from langgraph.graph import StateGraph, START, END
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace
from typing import TypedDict
from dotenv import load_dotenv

load_dotenv()

True

In [ ]:
llm = HuggingFaceEndpoint(
    repo_id="openai/gpt-oss-120b",
    task="text-generation"
)

model = ChatHuggingFace(llm=llm)

/mnt/d/Academics/Generative AI by CampusX/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
class BlogState(TypedDict):

    title: str
    outline: str
    content: str
    

In [4]:
def create_outline(state: BlogState) -> BlogState:
    
    # fetch title
    title = state['title']

    # call llm gen oultine
    prompt = f"Generate an  detailed outline for a blog on the topic - {title}"
    outline = model.invoke(prompt).content

    # update the state
    state['outline'] = outline

    return state

In [5]:
def create_blog(state: BlogState) -> BlogState:
    
    # fetch title and outline
    title = state['title']
    outline = state["outline"]

    # call llm gen oultine
    prompt = f"Generate an  detailed a blog on the topic - {title} using the following outline \n{outline}"
    content = model.invoke(prompt).content

    # update the state
    state['content'] = content

    return state

In [6]:
graph = StateGraph(BlogState)

# nodes
graph.add_node('create_outline', create_outline)
graph.add_node('create_blog', create_blog)

# edges
graph.add_edge(START, 'create_outline')
graph.add_edge('create_outline', 'create_blog')
graph.add_edge('create_blog', END)

# graph compile
workflow = graph.compile()
# workflow

In [18]:
# Execute the graph
initial_state = {'title': 'Rise of AI in India and suggestion for India'}

final_state = workflow.invoke(initial_state)

print(final_state)

HfHubHTTPError: 401 Client Error: Unauthorized for url: https://api.studio.nebius.ai/v1/chat/completions

In [14]:
print(final_state['outline'])

NameError: name 'final_state' is not defined

In [ ]:
print(final_state['content'])